In [1]:
import keras
from keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape
#from keras.layers import BatchNormalization
from keras.models import Model
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.7)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True, gpu_options=gpu_options))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6



In [3]:
import os
import PIL
import PIL.Image
import pathlib
from sklearn.model_selection import train_test_split
data_dir = pathlib.Path("/Users/Yernar-PC/DEEP LEARNING/PART2")
data_dir_train = pathlib.Path("/Users/Yernar-PC/DEEP LEARNING/Insulator/total data/all/*/*")

In [4]:
filenames = tf.io.gfile.glob(str(data_dir_train))

train_filenames, val_filenames = train_test_split(filenames, test_size=0.2)

In [5]:
COUNT_CLEAN = len([filename for filename in filenames if "Clean" in filename])
print("Clean images count in training set: " + str(COUNT_CLEAN))

COUNT_POLLUTED = len([filename for filename in filenames if "Polluted" in filename])
print("Polluted images count in training set: " + str(COUNT_POLLUTED))

Clean images count in training set: 71
Polluted images count in training set: 205


In [6]:
train_list_ds = tf.data.Dataset.from_tensor_slices(train_filenames)
val_list_ds = tf.data.Dataset.from_tensor_slices(val_filenames)

for f in train_list_ds.take(5):
    print(f.numpy())

b'\\Users\\Yernar-PC\\DEEP LEARNING\\Insulator\\total data\\all\\Polluted\\20200617_124128.jpg'
b'\\Users\\Yernar-PC\\DEEP LEARNING\\Insulator\\total data\\all\\Polluted\\20200617_124312.jpg'
b'\\Users\\Yernar-PC\\DEEP LEARNING\\Insulator\\total data\\all\\Clean\\20200617_123714.jpg'
b'\\Users\\Yernar-PC\\DEEP LEARNING\\Insulator\\total data\\all\\Polluted\\20200617_125832.jpg'
b'\\Users\\Yernar-PC\\DEEP LEARNING\\Insulator\\total data\\all\\Polluted\\DSCF0118.JPG'


In [7]:
TRAIN_IMG_COUNT = tf.data.experimental.cardinality(train_list_ds).numpy()
print("Train images count: " + str(TRAIN_IMG_COUNT))

VAL_IMG_COUNT = tf.data.experimental.cardinality(val_list_ds).numpy()
print("Validation images count: " + str(VAL_IMG_COUNT))

Train images count: 220
Validation images count: 56


In [8]:
CLASS_NAMES = np.array([str(tf.strings.split(item, os.path.sep)[-1].numpy())[2:-1]
                        for item in tf.io.gfile.glob(str(data_dir) + "/All_250/*")])
CLASS_NAMES

array([], dtype=float64)

In [9]:
def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    if parts[-2] == 'Clean':
        return 0
    else:
        return 1

In [79]:
def decode_img(img):
    # convert the compressed string
    img_X = tf.image.decode_jpeg(img, channels=3)
    
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    #img_X = tf.keras.layers.Rescaling(scale=1./255)(img_X)
    #img_X = tf.keras.layers.Rescaling(scale=1./127.5, offset = -1)(img_X)
    img_X = tf.cast(img_X, tf.float32)
    img_X = tf.keras.applications.nasnet.preprocess_input(img_X)
    #img_X = tf.keras.applications.mobilenet_v2.preprocess_input(img_X)
    #img_X = tf.keras.applications.xception.preprocess_input(img_X)
    #img_X = tf.keras.applications.resnet50.preprocess_input(img_X)
    #img_X = tf.keras.applications.densenet.preprocess_input(img_X)
    #img_X = tf.keras.applications.vgg16.preprocess_input(img_X)
    #img_X = tf.keras.applications.efficientnet.preprocess_input(img_X)
    # resize the image to the desired size.
    return tf.image.resize(img_X, IMAGE_SIZE)

In [80]:
def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [81]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMAGE_SIZE = (224, 224)
train_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

val_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [82]:
X_train = []
y_train = []

for image, label in train_ds:
    X_train.append(image)
    y_train.append(label)    
X_train = np.array(X_train, dtype = 'float32')
y_train = np.array(y_train, dtype = 'int8')

shuffled_idx = np.random.permutation(TRAIN_IMG_COUNT)
X_train[:] = X_train[shuffled_idx]
y_train[:] = y_train[shuffled_idx]
print(X_train.shape)
print(y_train.shape)

(220, 224, 224, 3)
(220,)


In [83]:
X_val = []
y_val = []

for image, label in val_ds:
    X_val.append(image)
    y_val.append(label)    
X_val = np.array(X_val, dtype = 'float32')
y_val = np.array(y_val, dtype = 'int8')

shuffled_idx = np.random.permutation(VAL_IMG_COUNT)
X_val[:] = X_val[shuffled_idx]
y_val[:] = y_val[shuffled_idx]
print(X_val.shape)
print(y_val.shape)

(56, 224, 224, 3)
(56,)


In [84]:
data_train = np.concatenate((X_train, X_val))
label_train = np.concatenate((y_train, y_val))

In [85]:
data_train[1][1]

array([[ 0.7092312 ,  0.7798194 ,  0.74060374],
       [ 0.7379877 ,  0.7850465 ,  0.753674  ],
       [ 0.741044  ,  0.7881028 ,  0.75673026],
       [ 0.69917715,  0.7697654 ,  0.7305497 ],
       [ 0.7096688 ,  0.78025705,  0.74104136],
       [ 0.7318628 ,  0.7789216 ,  0.76323533],
       [ 0.7176471 ,  0.7882353 ,  0.7490196 ],
       [ 0.7352168 ,  0.805805  ,  0.76658934],
       [ 0.7421794 ,  0.7892382 ,  0.77355194],
       [ 0.7453781 ,  0.7924369 ,  0.7767506 ],
       [ 0.71372575,  0.76078457,  0.7450983 ],
       [ 0.760294  ,  0.80735284,  0.79166657],
       [ 0.73589945,  0.78295827,  0.7515857 ],
       [ 0.73410356,  0.7954482 ,  0.7654761 ],
       [ 0.73105   ,  0.80163825,  0.77810884],
       [ 0.740966  ,  0.81155425,  0.78802484],
       [ 0.7219187 ,  0.79250693,  0.7689775 ],
       [ 0.736222  ,  0.80681026,  0.78328085],
       [ 0.7324225 ,  0.818697  ,  0.7873244 ],
       [ 0.72012055,  0.80639505,  0.7750225 ],
       [ 0.7244948 ,  0.8107693 ,  0.763

In [86]:
label_train[1]

1

## Concatenate the Large Dimensoinal Layers

#### 1. NASNetMobile

In [87]:
IMG_SHAPE = IMAGE_SIZE + (3, )
base_model_NASNetMobile = tf.keras.applications.NASNetMobile(input_shape=IMG_SHAPE,
                                                         include_top=False,
                                                         weights='imagenet')
feature_batch = base_model_NASNetMobile(data_train)
base_model_NASNetMobile.trainable = False
flatten_average_layer = tf.keras.layers.Flatten()
feature_batch_NASNetMobile = flatten_average_layer(feature_batch)
print(feature_batch_NASNetMobile.shape)

(276, 51744)


#### 2. MobileNet

In [78]:
IMG_SHAPE = IMAGE_SIZE + (3, )
base_model_MobileNet = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                         include_top=False,
                                                         weights='imagenet')
feature_batch = base_model_MobileNet(data_train)
base_model_MobileNet.trainable = False
flatten_average_layer = tf.keras.layers.Flatten()
feature_batch_MobileNet = flatten_average_layer(feature_batch)
print(feature_batch_MobileNet.shape)

(276, 62720)


#### 3. Xception

In [23]:
# Due to the error from out of memory, the the image sizes weremnchanged to 150 for Xception, Resnet50, Densenet201
IMG_SHAPE = IMAGE_SIZE + (3, )
base_model_Xception = tf.keras.applications.Xception(input_shape=IMG_SHAPE,
                                                         include_top=False,
                                                         weights='imagenet')
feature_batch = base_model_Xception(data_train)
base_model_Xception.trainable = False
flatten_average_layer = tf.keras.layers.Flatten()
feature_batch_Xception = flatten_average_layer(feature_batch)
print(feature_batch_Xception.shape)

(276, 51200)


#### 4. ResNet50

In [32]:
base_model_ResNet50 = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                                         include_top=False,
                                                         weights='imagenet')
feature_batch = base_model_ResNet50(data_train)
base_model_ResNet50.trainable = False
flatten_average_layer = tf.keras.layers.Flatten()
feature_batch_ResNet50 = flatten_average_layer(feature_batch)
print(feature_batch_ResNet50.shape)

(276, 51200)


#### 6. DenseNet201

In [41]:
IMG_SHAPE = IMAGE_SIZE + (3, )
base_model_DenseNet201 = tf.keras.applications.DenseNet201(input_shape=IMG_SHAPE,
                                                         include_top=False,
                                                         weights='imagenet')
feature_batch = base_model_DenseNet201(data_train)
base_model_DenseNet201.trainable = False
flatten_average_layer = tf.keras.layers.Flatten()
feature_batch_DenseNet201 = flatten_average_layer(feature_batch)
print(feature_batch_DenseNet201.shape)

(276, 30720)


#### 7. VGG16

In [50]:
# Due to the error from out of memory, the the image sizes weremnchanged to 120 for VGG16 and EfficientNetB7
IMG_SHAPE = IMAGE_SIZE + (3, )

base_model_VGG16 = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                                         include_top=False,
                                                         weights='imagenet')
feature_batch = base_model_VGG16(data_train)
base_model_VGG16.trainable = False
flatten_average_layer = tf.keras.layers.Flatten()
feature_batch_VGG16 = flatten_average_layer(feature_batch)
print(feature_batch_VGG16.shape)

(276, 4608)


#### 8. EfficientNetB7

In [59]:
base_model_EfficientNetB7 = tf.keras.applications.EfficientNetB7(input_shape=IMG_SHAPE,
                                                         include_top=False,
                                                         weights='imagenet')
feature_batch = base_model_EfficientNetB7(data_train)
base_model_EfficientNetB7.trainable = False
flatten_average_layer = tf.keras.layers.Flatten()
feature_batch_EfficientNetB7 = flatten_average_layer(feature_batch)
print(feature_batch_EfficientNetB7.shape)

(276, 40960)


##### Now we have to concatenate them

In [88]:
large_dim_input = np.concatenate((feature_batch_EfficientNetB7,feature_batch_VGG16,feature_batch_DenseNet201,
                                  feature_batch_NASNetMobile,feature_batch_ResNet50,feature_batch_Xception,
                                  feature_batch_MobileNet), axis = 1)
large_dim_input.shape

(276, 293152)

In [89]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
#from sklearn.utils.fixes import loguniform
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score



cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)

model = LogisticRegression(penalty = 'l1')

space = dict()
space['solver'] = ['saga', 'lbfgs', 'liblinear']
#space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
space['C'] = [0.001, 0.01, 0.1, 10, 100, 1000]

# define search
search = GridSearchCV(model, space, scoring = 'precision', n_jobs=1, cv=cv_inner, refit=True)

cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)

scores = cross_val_score(search, large_dim_input, label_train, cv=cv_outer, n_jobs=-1, error_score="raise")

In [90]:
from numpy import mean
from numpy import std
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.966 (0.038)
